### Global and Local Scopes

For the example `a = 10`, we say that the variable 'a' points to, or is bound to, the integer object. That object can accessed using the variable in various parts of our code...but not everywhere!

The variable name and its binding only exist in specific parts of our code.

The portion of code where the name/binding is defined is called the *lexical scope* of the variable.

These bindings are stored in __namespaces__, and each scope has an associated namespace.

__The Global Scope__

The global scope is essentially the module scope, and spans a single file only.

[ ! ] Global scopes are nested inside the built-in scope.

            |-----------------------(Built-in Scope)-----------------------|
                   (Module1 Scope)                     (Module2 Scope)
                   

If you reference a variable name inside a scope and Python does not find it in that scope's namespace, it will traverse upwards and look in an enclosing scopes namespace.

__The Local Scope__

Variables defined inside a function are not created until the function is called, and every time the function is called, a new scope is created.

Variables defined inside the function are assigned to that scope (the function local scope).

[ ! ] The actual object the variable references could be different each time the function called



In [1]:
# Here, a, b, and c are defined inside the function local scope, and will be created when
# the function is called.
def my_func(a, b):
    c = a * b
    return c

In [2]:
# a = 'z', b = 2, c = 'zz'
my_func('z', 2)

# a = 10, b = 5, c = 50
my_func(10, 5)

# The variable names are the same, yet they are stored in different scopes

50

__Nested Scopes__

            |-----------------------(Built-in Scope)-----------------------|
               |---------(Module Scope)---------|
                 (Local Scope)    (Local Scope)

__Namespace Lookups__

When requesting the object bound to a variable name: eg. `print(a)` ...

Python will try to find the object bound to the variable
- in the current local scope first
- then works up the chain of enclosing scopes

__Accessing the Global Scope from a Local Scope__

What if we modify a global variable from inside a function?

In [3]:
# Defined in global scope
a = 0

# Since we make an assigment to 'a' inside the function, Python will interpret 'a' as a 
# local variable to the function
def my_func():
    a = 100 # local var 'a' masks global variable 'a'
    print(a)

my_func()

print(a) #Will print 0, since this global variable was never modified by my_func

100
0


__`global`__

We can tell Python that a variable is meant to be scoped in the global scope by using the `global` keyword.

In [4]:
a = 0

def my_func():
    global a #This tells Python to access this variable from the global namespace.
    a = 100

my_func()

print(a) # Will print 100, since my_func modified 'a' from the global scope.

100


__Global and Local Scoping__

When Python encounters a function definition at compile-time, it will scan for any labels (variables) that have values __assigned__ to them anywhere in the function.

If the label has not been specified as global, then it will be local.

Variables that are referenced but __not assigned__ a value anywhere in the function will not be local, and Python will, at run-time, look for them in enclosing scopes.

In [5]:
a = 10

def func1():
    print(a) # Since 'a' is only referenced, its treated as nonlocal
    
def func2():
    a = 100 # Assignment, therefore treated as local

def func3():
    global a # Specified global, will treat as nonlocal
    a = 100
    
def func4():
    print(a) #Assignment elsewhere in function, treated as local but not defined -> Exception
    a = 100 

### Non-Local Scopes

__Inner Functions__

We can define functions inside another function.

In [1]:
def outer_func():
    
    def inner_func():
        pass
    
    inner_func()
    
outer_func()

Both functions have acess to the global and built-in scopes as well as their respective local scopes. But the *inner* function also has access to its enclosing scope - the scope of the *outer* function. This scope is called a __non-local__ scope.

__Modifying global Variables__

In [3]:
a = 10

def outer_func():
    def inner_func():
        global a
        a = 'hello'
        
    inner_func()
    
outer_func()
print(a)

hello


__Modifying nonlocal Variables__

In [6]:
def outer_func():
    x = 'hello'
    
    #Since 'x' is assigned to in inner_func, and therefore treated as a local var, 
    #we must specify that its nonlocal
    def inner_func():
        nonlocal x
        x = 'python'
        
    inner_func()
    print(x)
    
outer_func()

python


__Nonlocal Variables__

Whenever Python is told a variable is `nonlocal`, it will look for it in the *enclosing local scopes chain* until it first encounters the variable name.

__Beware:__ It will only look in local scopes, it will NOT look in the global scope

### Closures

__Free Variables and Closures__

In [8]:
def outer():
    x = 'python'
    
    # When inner is created, before it is called, it will enclose the variable 'x'.
    # The value of 'x' will be evaluated when the function is called.
    # So if we return this function, it will retain its access to 'x' and can be accessed
    # when the returned function is called. This is a closure.
    def inner():
        # This x refers to the one in outer's scope. This nonlocal variable is called
        # a 'free variable'.
        print(f"{x} rocks!")
        
    inner()
    
outer()

python rocks!


__Returning the inner Function__

In [11]:
def outer():
    x = 'python'
    def inner():
        print(f"{x} rocks!")
    
    return inner

In [13]:
encloses_x = outer()

# This function call still has access to 'x', even though outer has already finished running
# and its scope is gone.
encloses_x()

python rocks!


__Python Cells and Multi-Scoped Variables__

Here, the value of `x` is shared between two scopes: *outer* and the *inner* closure

```
def outer():
    x = 'python'
    def inner():
        print(x)
    return inner
```

The label `x` is in two different scopes, the always references the same value.

Python does this by creating a __cell__ as an intermediary object.

outer scope(x) --->

                    cell 0x100 ---> memory address 0x200 ---> 'python'
                    
inner scope(x) --->

In effect, both variables 'x' point to the same *cell*. When requesting the value of the variable, Python does a "double-hop" to the final value stored at the memory address.

__Closures__

You can think of closures as a function plus an extended scope that the contains free varaibles. The free varaibles value is the object the cell points to - so that could change!

__Closure Introspection__

In [14]:
def outer():
    x = 10
    y = 'python'
    def inner():
        x = 10
        print(y)
    return inner

In [15]:
fn = outer()

fn.__code__.co_freevars

('y',)

In [16]:
fn.__closure__

(<cell at 0x7faf55793850: str object at 0x7faf719b2c30>,)

__Modifying Free Variables__

In [17]:
def counter():
    count = 0
    
    def inc():
        nonlocal count
        count += 1
        return count
    
    return inc

In [18]:
inc_counter = counter()

inc_counter()

1

In [20]:
inc_counter()

2

__Multiple Instances of Closures__

Every time we run a function, a new scope is created. If that function generates a closure, a new closure is created every time as well.

In [21]:
f1 = counter()
f2 = counter()

f1()
f1()

2

In [22]:
f2()

1

__Shared Extended Scopes__

In [23]:
def outer():
    count = 0
    
    def inner1():
        nonlocal count
        count += 1
        return count
    def inner2():
        nonlocal count
        count += 1
        return count
    
    return inner1, inner2

In [25]:
f1, f2 = outer()

f1() # Increments 0 to 1
f2() # Increments the same object, so increments 1 to 2

2

__Nested Closures__

In [31]:
def incrementer(step):
    # incrementer returns the closure of inner + freevar of step
    def inner(start):
        # inner returns the closure of inc + freevars step and current
        current = start
        def inc():
            nonlocal current
            current += step
            return current
        
        return inc
    return inner

In [32]:
add_2 = incrementer(2)

inc = add_2(100)
inc()
inc()

104

### Decorators

In general, a *decorator* function:
- takes a function as an argument
- returns a closure
- the closure usually accepts any combination of parameters
- runs some code in the inner function (closure)
- the closure function calls the passed in function using the arguments passed to the closure
- returns whatever is returned by that function call

In [2]:
def counter(fn):
    count = 0
    
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f"Function {fn.__name__} was called {count} times.")
        return fn(*args, **kwargs)
    return inner

In [3]:
def add(a, b=0):
    return a + b

In [4]:
# We are essentially modifying out add function by wrapping it inside another
# function that adds some functionality to it.
add = counter(add)

res = add(1, 2)
print(res)

Function add was called 1 times.
3


__Decorators and the `@` Symbol__


In [7]:
# We can decorate a function using the explicit way:
def add(a, b):
    return a + b

add = counter(add)

# Or we can use Python's @ symbol to shortcut that process
@counter
def add(a, b):
    return a + b

__Introspecting Decorated Functions__

In [8]:
@counter
def mult(a, b, c=1):
    """ Returns the product of three values
    """
    return a * b * c

In [9]:
# Since we actually re-assign the label 'mult' to the decorator function,
# the function object actually points to the 'inner' function from counter.
mult.__name__

'inner'

In [10]:
help(mult) # we lose the function signature as well as docstrings

Help on function inner in module __main__:

inner(*args, **kwargs)



__The `functools.wraps` function__

The functools module has a `wraps` function, which is itself a decorator, that we can use to fix the metadata of our inner function in our decorator.

In [11]:
from functools import wraps

In [12]:
def counter(fn):
    count = 0
    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f"Function {fn.__name__} was called {count} times.")
        return fn(*args, **kwargs)
    return inner

In [14]:
@counter
def mult(a, b, c=1):
    """ Returns the product of three values
    """
    return a * b * c

In [15]:
help(mult)

Help on function mult in module __main__:

mult(a, b, c=1)
    Returns the product of three values



__Decorator Factories Or Decorators with Parameters__

The `timed_with_reps` function below is itself not a decorator, instead it *returns* a decorator when called. Any arguments passed to `timed_with_reps` can be referenced insdie the decorator.

In [21]:
def timed_with_reps(reps):
    def timed_decorator(fn):
        from time import perf_counter
        @wraps(fn)
        def inner(*args, **kwargs):
            total_elapsed = 0

            for i in range(reps): # reps is a free variable from timed_with_reps
                start = perf_counter()
                result = fn(*args, **kwargs)
                total_elapsed += (perf_counter() - start)

            avg_elapsed = total_elapsed / reps
            print(avg_elapsed)
            return result
        
        return inner
    # calling timed_with_reps(n) returns the original decorator with reps set to n
    return timed_decorator

In [22]:
@timed_with_reps(10)
def my_func(): ...